# AEM data inversion widget 
### Interactive widget for testing how well the AEM method can resolve subsurface targets 
<br>

In [1]:
import sys, os
from SimPEG import *
from simpegEM1D import (
    EM1D, EM1DSurveyTD, Utils1D, get_vertical_discretization_time, 
    set_mesh_1d, skytem_HM_2015
)
import numpy as np
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
# %pylab inline

In [2]:
#Setup
from matplotlib.patches import Rectangle

time = np.logspace(-5, -2, 31)
hz = get_vertical_discretization_time(time, facter_tmax=0.3, factor_tmin=10., n_layer=19)
mesh1D = set_mesh_1d(hz)
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC
TDsurvey = EM1DSurveyTD(
    rx_location = np.array([0., 0., 100.+30.]),
    src_location = np.array([0., 0., 100.+30.]),
    topo = np.r_[0., 0., 100.],
    depth = depth,
    rx_type = 'dBzdt',
    wave_type = 'stepoff',
    src_type = 'CircularLoop',
    a = 13.,
    I = 1.,
    time = time,
    base_frequency = 25.,
    use_lowpass_filter=False,
    high_cut_frequency=210*1e3        
)
sig_half = 1./20.
sig_blk = sig_half * 20.
chi_half = 0.
expmap = Maps.ExpMap(mesh1D)


def define_sigma_layers(sig_half,sig_lay,layers):
    sig  = np.ones(TDsurvey.n_layer)*sig_half
    for i in range(len(layers)):
        if i==len(layers)-1:
            ind = (layers[i]>LocSigZ)
        else:
            ind = np.where((layers[i]>LocSigZ) & (layers[i+1]<=LocSigZ))
    #     print(i,sig_lay[i],ind)
        sig[ind] = sig_lay[i]
    return sig
        
    
def plot_mesh(sig,mesh1D):
    fig, ax = plt.subplots(1,1, figsize=(5, 8))
    Utils1D.plotLayer(sig, mesh1D, showlayers=True)
    ax.set_ylim(-50,350)
    xlims = ax.get_xlim()
    rect = Rectangle((xlims[0],-50),xlims[1]-xlims[0],50,fc=[.8,.8,.8])
    ax.annotate('Air', (.5, .94), color='k', weight='normal', 
            fontsize=20,xycoords='axes fraction', ha='center', va='center')
#     plt.annotate()
    ax.add_patch(rect)
    ax.invert_yaxis()
    return fig,ax

def prob_and_run(w):
    #Problem
    m_true = np.log(w.result)
    prob = EM1D(mesh1D, sigmaMap=expmap, verbose=False)
    if prob.ispaired:
        prob.unpair()
    if TDsurvey.ispaired:
        TDsurvey.unpair()
    prob.pair(TDsurvey)
    prob.chi = np.zeros(TDsurvey.n_layer)
    d_true = TDsurvey.dpred(m_true)


    #Data and noise 
    np.random.seed(1)
    TDsurvey.dtrue = d_true
    std = 0.05
    noise = std*abs(TDsurvey.dtrue)*np.random.randn(*TDsurvey.dtrue.shape)
    floor = 0.
    std = 0.07
    TDsurvey.dobs = TDsurvey.dtrue+noise
    uncert = abs(TDsurvey.dobs)*std+floor

    dmisfit = DataMisfit.l2_DataMisfit(TDsurvey)
    uncert = (abs(TDsurvey.dobs)*std+floor)
    dmisfit.W = 1./ uncert

    #Starting model
    m0 = np.log(np.ones_like(w.result)*w.kwargs['sig_half'])
    d_0 = TDsurvey.dpred(m0)

    reg = Regularization.Sparse(
        mesh1D,
        mapping=Maps.IdentityMap(mesh1D),
        alpha_s=1.,
        alpha_x=1.
    )
    p = 0
    qx, qz = 2., 2.
    reg.norms = np.c_[p, qx, qz, 0.]
    IRLS = Directives.Update_IRLS(
        maxIRLSiter=20, minGNiter=1, fix_Jmatrix=True, coolingRate=2, betaSearch=False,
        chifact_start = 1.
    )
    opt = Optimization.ProjectedGNCG(maxIter = 25)
    invProb = InvProblem.BaseInvProblem(dmisfit, reg, opt)
    beta = Directives.BetaSchedule(coolingFactor=2., coolingRate=1)
    betaest = Directives.BetaEstimate_ByEig(beta0_ratio=1e0)
    target = Directives.TargetMisfit()
    # update_sense = Directives.UpdateSensitivityWeights(threshold=delta)
    # inv = Inversion.BaseInversion(invProb, directiveList=[IRLS, betaest])
    inv = Inversion.BaseInversion(invProb, directiveList=[beta, betaest, target])
    prob.counter = opt.counter = Utils.Counter()
    opt.LSshorten = 0.5
    opt.remember('xc')

    #Run
    mopt = inv.run(m0)
    return mopt


def plot_results(mopt):
    fig, ax = subplots(1,1, figsize=(5, 8))
    Utils1D.plotLayer(w.result, mesh1D)
    Utils1D.plotLayer(expmap*mopt, mesh1D, showlayers=True, **{'color':'r'})
    # Utils1D.plotLayer(expmap*invProb.l2model, mesh1D, showlayers=False, **{'color':'b', 'lw':1.})
    ax.set_ylim(-50,350)
    xlims = ax.get_xlim()
    rect = Rectangle((xlims[0],-50),xlims[1]-xlims[0],50,fc=[.8,.8,.8])
    ax.annotate('Air', (.5, .94), color='k', weight='normal', 
            fontsize=20,xycoords='axes fraction', ha='center', va='center')
#     plt.annotate()
    ax.add_patch(rect)
    ax.invert_yaxis()
    return fig,ax

# decorater used to block function printing to the console
def blockPrinting(func):
    def func_wrapper(*args, **kwargs):
        # block all printing to the console
        sys.stdout = open(os.devnull, 'w')
        # call the method in question
        value = func(*args, **kwargs)
        # enable all printing to the console
        sys.stdout = sys.__stdout__
        # pass the return value of the method back
        return value

    return func_wrapper


# sig_half= 1./20.
# layers  = -1.*np.r_[0,30,70,100]  
# sig_lay     = np.r_[1.,1./20,1./100,sig_half]

# sig = define_sigma_layers(sig_half,sig_lay,layers)
# f,ax = plot_mesh(sig,mesh1D)

## Step 1. Enter the subsurface resistivity structure 

In [3]:
from ipywidgets import BoundedFloatText, interactive,HTML
from IPython.display import display

lay1_box =  BoundedFloatText(
                            value=30,
                            min=0,
                            max=1000.0,
                            step=0.1,
                            description='Lay1 depth:',
                            disabled=False
)

sig1_box =  BoundedFloatText(
                            value=.001,
                            min=1e-10,
                            max=1e10,
                            step=0.1,
                            description='Lay1 cond.',
                            disabled=False
)

lay2_box =  BoundedFloatText(
                            value=70,
                            min=0,
                            max=1000.0,
                            step=0.1,
                            description='Lay2 depth:',
                            disabled=False
)

sig2_box =  BoundedFloatText(
                            value=.1,
                            min=1e-10,
                            max=1e10,
                            step=0.1,
                            description='Lay2 cond.',
                            disabled=False
)

sighalf_box =  BoundedFloatText(
                            value=.01,
                            min=1e-10,
                            max=1e10,
                            step=0.1,
                            description='Back. cond.:',
                            disabled=False
)



def f(sig1,lay1,sig2,lay2,sig_half=sighalf_box):
    layers = -np.r_[0,lay1,lay2]
    sig_lay = np.r_[sig1,sig2,sig_half]
    sig = define_sigma_layers(sig_half,sig_lay,layers)
    plot_mesh(sig,mesh1D)
    plt.show()
#     print(sig1,lay1)
    return sig

    
w = interactive(f, sig1=sig1_box, lay1=lay1_box,sig2=sig2_box, lay2=lay2_box)
w

interactive(children=(BoundedFloatText(value=0.001, description='Lay1 cond.', max=10000000000.0, min=1e-10, st…

## Step 2. Once the subsurface resistivity structure is entered, click below to run the inverison 

In [58]:
import ipywidgets as widgets
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from IPython.display import clear_output,display

button = widgets.Button(description="Run inversion!")
output = widgets.Output()
display(button, output)

@blockPrinting
def on_button_clicked(b):
    print('Running inversion...')
    with output:
        clear_output()
        mopt = prob_and_run(w)
        f,ax = plot_results(mopt)
        show_inline_matplotlib_plots()
    return

button.on_click(on_button_clicked)


Button(description='Run inversion!', style=ButtonStyle())

Output()